In [10]:
# Import the required module for speech recognition
import speech_recognition as sr # %pip install Speechrecognition


# Define the main function
def main() :

    # Create a recognizer object using the speech recognition library
    r = sr.Recognizer()

    # Open the microphone and create a microphone object using the speech recognition library
    with sr.Microphone() as source :

        # Adjust the recognizer sensitivity to ambient noise levels
        r.adjust_for_ambient_noise(source)

        # Prompt the user to say something
        print ("say something..")

        # Capture the audio and store it in the 'audio' variable
        audio = r.listen(source)

        
        print("Recognizing Now .... ")

        # Use the Google Speech Recognition API to convert the spoken language into text
        try :

            # Print the recognized text
            print("You have said :" + r.recognize_google(audio))

        except Exception as e :
            
            # If an error occurs, print the error message
            print("Error :" + str(e))

# Call the main function when the script is run directly
if __name__ == "__main__" :
    main()

say something..
Recognizing Now .... 
Error :


In [6]:
import speech_recognition as sr


def main():

    r = sr.Recognizer()

    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)

        print("Please say something")

        audio = r.listen(source)

        print("Recognizing Now .... ")


        # recognize speech using google

        try:
            print("You have said \n" + r.recognize_google(audio))
            print("Audio Recorded Successfully \n ")


        except Exception as e:
            print("Error :  " + str(e))




        # write audio
        with open("recorded.wav", "wb") as f:
            f.write(audio.get_wav_data())


if __name__ == "__main__":
    main()

Please say something
Recognizing Now .... 
You have said 
hello we are CS 219 implement an interview simulator hello
Audio Recorded Successfully 
 


import ipywidgets as widgets
from IPython.display import display
from queue import Queue
from threading import Thread

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description='Record',
    disabled=False,
    button_style='success',
    tooltip='Record',
    icon='microphone'
)

stop_button = widgets.Button(
    description='Stop',
    disabled=False,
    button_style='warning',
    tooltip='Stop',
    icon='stop'
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)
    
    with output:
        display("Starting...")
        
        record = Thread(target= record_microphone)
        record.start()
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
    
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

# Install pyaudio from http://people.csail.mit.edu/hubert/pyaudio/
# Find audio device index using this code
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()


CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
   
        
        p = pyaudio.PyAudio()

        stream = p.open(format=AUDIO_FORMAT,
                        channels=CHANNELS,
                        rate=FRAME_RATE,
                        input=True,
                        input_device_index=1,
                        frames_per_buffer=chunk)

        frames = []

        while not messages.empty():
            data = stream.read(chunk)
            frames.append(data)
            if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
                recordings.put(frames.copy())
                frames = []

        stream.stop_stream()
        stream.close()
        p.terminate()
        

    

         
  


        

import subprocess
import json
from vosk import Model, KaldiRecognizer
import time

model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)
    
def speech_recognition(output):
    
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)
        output.append_stdout(cased)
        time.sleep(1)
